In [1]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np
from sklearn.base import BaseEstimator,TransformerMixin, ClassifierMixin
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import ElasticNetCV, LassoLarsCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline, make_union
from sklearn.utils import check_array
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import r2_score

In [16]:
class StackingEstimator(BaseEstimator, TransformerMixin):
    
    def __init__(self, estimator):
        self.estimator = estimator

    def fit(self, X, y=None, **fit_params):
        self.estimator.fit(X, y, **fit_params)
        return self
    def transform(self, X):
        X = check_array(X)
        X_transformed = np.copy(X)
        # add class probabilities as a synthetic feature
        if issubclass(self.estimator.__class__, ClassifierMixin) and hasattr(self.estimator, 'predict_proba'):
            X_transformed = np.hstack((self.estimator.predict_proba(X), X))

        # add class prodiction as a synthetic feature
        X_transformed = np.hstack((np.reshape(self.estimator.predict(X), (-1, 1)), X_transformed))

        return X_transformed

In [17]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
target = train.y
test_ids = test.ID
train.drop(['ID','y'], axis=1,  inplace= True)
test.drop(['ID'], axis = 1, inplace= True)

In [18]:
print (train.shape, test.shape)

(4209, 376) (4209, 376)


In [19]:
train.head()

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,k,v,at,a,d,u,j,o,0,0,...,0,0,1,0,0,0,0,0,0,0
1,k,t,av,e,d,y,l,o,0,0,...,1,0,0,0,0,0,0,0,0,0
2,az,w,n,c,d,x,j,x,0,0,...,0,0,0,0,0,0,1,0,0,0
3,az,t,n,f,d,x,l,e,0,0,...,0,0,0,0,0,0,0,0,0,0
4,az,v,n,f,d,h,d,n,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
train_test_data = train.append(test)

In [21]:
for f in train_test_data.columns.tolist():
    if (train_test_data[f].dtype=='object'):
        train_test_data[f]  =  pd.factorize(train_test_data[f])[0]

In [22]:
train_test_data['X5_8'] = train_test_data['X5'] + train_test_data['X8']
train_test_data['X5_1'] = train_test_data['X5'] + train_test_data['X1']
train_test_data['X5_6'] = train_test_data['X5'] + train_test_data['X6']
train_test_data['X5_3'] = train_test_data['X5'] + train_test_data['X3']
train_test_data['X5_2'] = train_test_data['X5'] + train_test_data['X2']
train_test_data['X5_0'] = train_test_data['X5'] + train_test_data['X0']
train_test_data['X8_1'] = train_test_data['X8'] + train_test_data['X1']
train_test_data['X8_6'] = train_test_data['X8'] + train_test_data['X6']
train_test_data['X8_3'] = train_test_data['X8'] + train_test_data['X3']
train_test_data['X8_2'] = train_test_data['X8'] + train_test_data['X2']
train_test_data['X8_0'] = train_test_data['X8'] + train_test_data['X0']
train_test_data['X1_6'] = train_test_data['X1'] + train_test_data['X6']
train_test_data['X1_3'] = train_test_data['X1'] + train_test_data['X3']
train_test_data['X1_2'] = train_test_data['X1'] + train_test_data['X2']
train_test_data['X1_0'] = train_test_data['X1'] + train_test_data['X0']
train_test_data['X6_3'] = train_test_data['X6'] + train_test_data['X3']
train_test_data['X6_2'] = train_test_data['X6'] + train_test_data['X2']
train_test_data['X6_0'] = train_test_data['X6'] + train_test_data['X0']
train_test_data['X3_2'] = train_test_data['X3'] + train_test_data['X2']
train_test_data['X3_0'] = train_test_data['X3'] + train_test_data['X0']
train_test_data['X2_0'] = train_test_data['X2'] + train_test_data['X0']

train_test_data.head(3)

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,...,X1_6,X1_3,X1_2,X1_0,X6_3,X6_2,X6_0,X3_2,X3_0,X2_0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,1,0,1,1,0,0,0,...,2,2,2,1,2,2,1,2,1,1
2,1,2,2,2,0,2,0,1,0,0,...,2,4,4,3,2,2,1,4,3,3


In [29]:
train= train_test_data.iloc[:train.shape[0]]
test= train_test_data.iloc[train.shape[0]:]

In [30]:
n_comp = 12

# tSVD
tsvd = TruncatedSVD(n_components=n_comp, random_state=420)
tsvd_results_train = tsvd.fit_transform(train)
tsvd_results_test = tsvd.transform(test)

# PCA
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train = pca.fit_transform(train)
pca2_results_test = pca.transform(test)

# ICA
ica = FastICA(n_components=n_comp, random_state=420)
ica2_results_train = ica.fit_transform(train)
ica2_results_test = ica.transform(test)

# GRP
grp = GaussianRandomProjection(n_components=n_comp, eps=0.1, random_state=420)
grp_results_train = grp.fit_transform(train)
grp_results_test = grp.transform(test)

# SRP
srp = SparseRandomProjection(n_components=n_comp, dense_output=True, random_state=420)
srp_results_train = srp.fit_transform(train)
srp_results_test = srp.transform(test)

#save columns list before adding the decomposition components

usable_columns = list(set(train.columns) - set(['y']))

# Append decomposition components to datasets
for i in range(1, n_comp + 1):
    train['pca_' + str(i)] = pca2_results_train[:, i - 1]
    test['pca_' + str(i)] = pca2_results_test[:, i - 1]

    train['ica_' + str(i)] = ica2_results_train[:, i - 1]
    test['ica_' + str(i)] = ica2_results_test[:, i - 1]

    train['tsvd_' + str(i)] = tsvd_results_train[:, i - 1]
    test['tsvd_' + str(i)] = tsvd_results_test[:, i - 1]

    train['grp_' + str(i)] = grp_results_train[:, i - 1]
    test['grp_' + str(i)] = grp_results_test[:, i - 1]

    train['srp_' + str(i)] = srp_results_train[:, i - 1]
    test['srp_' + str(i)] = srp_results_test[:, i - 1]


#finaltrainset and finaltestset are data to be used only the stacked model (does not contain PCA, SVD... arrays) 
finaltrainset = train[usable_columns].values
finaltestset = test[usable_columns].values

In [31]:
'''Train the xgb model then predict the test data'''

xgb_params = {
    'n_trees': 520, 
    'eta': 0.0045,
    'max_depth': 4,
    'subsample': 0.93,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': np.mean(target), # base prediction = mean(target)
    'silent': 1
}

In [32]:
def xgb_r2(preds, dtrain):
    labels = dtrain.get_label()
    score = r2_score(labels, preds)
    return 'r2-score', score

In [33]:
X, y = train, target
dtrain = xgb.DMatrix(X, label= y )
dtest = xgb.DMatrix(test)
num_rounds = 1250
cv_output = xgb.cv(xgb_params, dtrain, feval= xgb_r2, num_boost_round=num_rounds, \
                   early_stopping_rounds=50, maximize=True, verbose_eval=100, show_stdv=False)

[0]	train-r2-score:0.00484	test-r2-score:0.00350667
[100]	train-r2-score:0.352083	test-r2-score:0.33767
[200]	train-r2-score:0.4995	test-r2-score:0.472611
[300]	train-r2-score:0.564823	test-r2-score:0.526334
[400]	train-r2-score:0.599257	test-r2-score:0.547282
[500]	train-r2-score:0.628505	test-r2-score:0.555426
[600]	train-r2-score:0.651492	test-r2-score:0.557664


In [34]:
num_rounds = len(cv_output)
watchlist = [(dtrain, 'dtrain')]
clf_xgb_main = xgb.train(dtrain=dtrain, params=xgb_params, feval= xgb_r2, maximize= True, num_boost_round=num_rounds,\
                         evals=watchlist, verbose_eval=100)
y_pred = clf_xgb_main.predict(dtest)

[0]	dtrain-r2-score:0.005187
[100]	dtrain-r2-score:0.348655
[200]	dtrain-r2-score:0.493135
[300]	dtrain-r2-score:0.555911
[400]	dtrain-r2-score:0.585423
[500]	dtrain-r2-score:0.609556
[600]	dtrain-r2-score:0.629316


In [37]:
'''Train the stacked models then predict the test data'''

stacked_pipeline = make_pipeline(
    StackingEstimator(estimator=LassoLarsCV(normalize=True)),
    StackingEstimator(estimator=GradientBoostingRegressor(learning_rate=0.001, loss="huber", max_depth=3, \
            max_features=0.55, min_samples_leaf=18, min_samples_split=14, subsample=0.7)), LassoLarsCV() )

stacked_pipeline.fit(finaltrainset, target)
results = stacked_pipeline.predict(finaltestset)

'''R2 Score on the entire Train data when averaging'''

print('R2 score on train data:')
print(r2_score(target,stacked_pipeline.predict(finaltrainset)*0.2855 + clf_xgb_main.predict(dtrain)*0.7145))

R2 score on train data:
0.620262545055


In [39]:
'''Average the predition on test data  of both models then save it on a csv file'''

sub = pd.DataFrame()
sub['ID'] = test_ids
sub['y'] = y_pred*0.75 + results*0.25

In [128]:
sub.to_csv('stacked-models_features_10thJuly1.csv', index=False)